In [1]:
#!pip install triton
import torch
import triton
import triton.language as tl
DEVICE = triton.runtime.driver.active.get_active_torch_device()
  



# GPU meaning: Many threads run together in lockstep groups on an SM.

@triton.jit
def add_kernel(x_ptr, n_elements, BLOCK: tl.constexpr):
    pid = tl.program_id(0)
    offsets = pid * BLOCK + tl.arange(0, BLOCK)
    mask = offsets < n_elements
    vals = tl.load(x_ptr + offsets, mask=mask, other=0.0)
    tl.device_print('pid=', pid, ' first_val=', vals[0])

size = 98432
x = torch.rand(size, device=DEVICE)
BLOCK = 128
grid = (triton.cdiv(size, BLOCK),)
add_kernel[grid](x, size, BLOCK=BLOCK)


In [ ]:
# GPU meaning: Many threads run together in lockstep groups on an SM.
size = 98432
x_ptr = torch.rand(size, device=DEVICE)

@triton.jit
def add_kernel(x_ptr):

    pid = tl.program_id(0)
    offsets=pid * 128 + tl.arange(0, 128)
    tl.device_print('pid=', offsets)
    x=tl.load(x_ptr+offsets)
    y=x*2
    tl.store(y)
                                 

add_kernel[(4,)](x_ptr)



triton.language.load(pointer, mask=None) -> Return a tensor of data whose values are loaded from memory at location defined by pointer.

1. If pointer is a single element pointer, a scalar is be loaded.
2. If pointer is an N-dimensional tensor of pointers, an N-dimensional tensor is loaded.
3. If pointer is a block pointer defined by make_block_ptr, a tensor is loaded. 




triton.language.store(pointer, value) -> Store a tensor of data into memory locations defined by pointer.

1. If pointer is a single element pointer, a scalar is stored
2. If pointer is an N-dimensional tensor of pointers, an N-dimensional block is stored. 
3. If pointer is a block pointer defined by make_block_ptr, a block of data is stored. 

In [ ]:
@triton.jit
def double(x_ptr,y_ptr):
    pid=tl.program_id(0)
    offsets=pid*128+tl.arange(0,128) # a list of pointers 
    x=tl.load(x_ptr+offsets)
    y=2*x
    tl.store(y_ptr+offsets,y)

Great question. The intent is:

pid * BLOCK_SIZE gives the start index of the chunk this program should handle.

pid = which program instance am I? (0, 1, 2, ...)
BLOCK_SIZE = how many elements each program handles
So each program gets a non-overlapping range.

Example with BLOCK_SIZE = 8:

pid=0 -> start = 0*8 = 0 -> handles indices 0..7
pid=1 -> start = 1*8 = 8 -> handles indices 8..15
pid=2 -> start = 2*8 = 16 -> handles indices 16..23
